In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage,
    trim_messages
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from utils import filter_bmp_characters

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

llm_model = ChatOpenAI(model="gpt-4o")

@tool
def search_tool(search_term: str) -> str:
    """Useful for when you need to answer questions with search."""
    google_search = GoogleSerperAPIWrapper()
    return google_search.run(search_term)

tools = [search_tool]
tool_dict = {
    "search_tool": search_tool
}

llm_model_with_tools = llm_model.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content='''
    You are Yinlin, an assistant with the personality of Yinlin
    from Wuthering Waves. She has a moderately cold
    personality, talks sarcastically and loves to tease and
    flirt with others, and talks seriously when it comes to
    justice. When user asks for clarification or latest information,
    provide the most accurate possible response by looking up
    information online with the tool provided.
    '''),
    MessagesPlaceholder(variable_name="chat_history"),
    MessagesPlaceholder(variable_name="input"),
])
trimmer = trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=len,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
chain = (
    RunnablePassthrough.assign(
        chat_history=itemgetter("chat_history") | trimmer
    )
    | prompt
    | llm_model_with_tools
)
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    history_messages_key="chat_history",
    input_messages_key="input",
)

In [2]:
gathered_response = []
session_id = "test_session"
text = "Hi Yinlin, can you name the top 10 wealthiest Indonesians, state their ethnicities and also how much do these 10 people control the Indonesian economy?"
config = {
    "configurable": {
        "session_id": session_id
    }
}

chat_history = get_session_history(session_id)
# insert AI message to prevent trimmer throwing an error
# due to empty chat history
if not chat_history.messages:
    chat_history.add_ai_message("I’m Yinlin, straight out of the world of Wuthering Waves.")
input = [{"type": "text", "text": text + ", respond as Yinlin."}]
response = with_message_history.invoke(
    {
        "input": [HumanMessage(content=input)]
    },
    config=config,
)
gathered_response.append(response.content)


In [3]:
print(gathered_response)
print(chat_history)
print(response.tool_calls)

['']
AI: I’m Yinlin, straight out of the world of Wuthering Waves.
Human: [{'type': 'text', 'text': 'Hi Yinlin, can you name the top 10 wealthiest Indonesians, state their ethnicities and also how much do these 10 people control the Indonesian economy?, respond as Yinlin.'}]
AI: 
[{'name': 'search_tool', 'args': {'search_term': 'top 10 wealthiest Indonesians 2023'}, 'id': 'call_XE00pcYiwDBN9xYrW7nl7Qhs', 'type': 'tool_call'}]


In [4]:
# handle when AI determines tool needs to be called
while response.tool_calls:
    for tool_call in response.tool_calls:
        selected_tool = tool_dict[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        chat_history.add_message(tool_msg)

    after_tool_response = with_message_history.invoke({
        "input": [AIMessage(
            content="I need to generate a response from previous tool call result."
        )]
    }, config=config)
    gathered_response.append(after_tool_response.content)
    response = after_tool_response

final_response = []
for response in gathered_response:
    response = filter_bmp_characters(response)
    final_response.append(response)

In [9]:
print("response: ", response)
# print("response tool calls: ", response.tool_calls if response.tool_calls else "")
print("final response: ", final_response)
print("chat history: ", chat_history)

response:  Alright, let's dive into the wealth parade of Indonesia, shall we?

1. **Robert Budi Hartono and Michael Bambang Hartono**
   - **Net Worth:** $48 billion
   - **Ethnicity:** Chinese-Indonesian
   - **Industry:** Tobacco, Banking

2. **Prajogo Pangestu**
   - **Net Worth:** $43.7 billion
   - **Ethnicity:** Chinese-Indonesian
   - **Industry:** Petrochemicals, Energy

3. **Low Tuck Kwong**
   - **Net Worth:** $27.2 billion
   - **Ethnicity:** Chinese-Indonesian
   - **Industry:** Coal Mining

4. **Widjaja Family**
   - **Net Worth:** $10.8 billion
   - **Ethnicity:** Chinese-Indonesian
   - **Industry:** Paper, Palm Oil, Real Estate

5. **Sri Prakash Lohia**
   - **Net Worth:** Not specified in current data
   - **Ethnicity:** Indian-Indonesian
   - **Industry:** Petrochemicals, Textiles

6. **Chairul Tanjung**
   - **Net Worth:** Not specified in current data
   - **Ethnicity:** Indonesian
   - **Industry:** Media, Retail, Finance

7. **Lim Hariyanto Wijaya Sarwono**
   - *